In [1]:
#Importación de librerías Requeridas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import numpy as np
import sys
import tensorflow as tf
import time
from sklearn.preprocessing import OneHotEncoder



In [2]:
import pandas as pd
file_path = 'banklimpio.csv'
X_cleaned1 = pd.read_csv(file_path)
X_cleaned1 = pd.read_csv(file_path, sep=',')  
X_cleaned1

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_management,job_retired,...,contact_telephone,contact_unknown,month_aug,month_feb,month_jul,month_jun,month_may,month_nov,poutcome_unknown,y_yes
0,58,2143,5,261,1,-1,0,0,1,0,...,0,1,0,0,0,0,1,0,1,0
1,44,29,5,151,1,-1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,33,2,5,76,1,-1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,47,1506,5,92,1,-1,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
4,33,1,5,198,1,-1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41114,23,113,17,266,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
41115,73,2850,17,300,1,40,8,0,0,1,...,0,0,0,0,0,0,0,1,0,1
41116,25,505,17,386,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
41117,71,1729,17,456,2,-1,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


Verificar Multicolinealidad

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Añadir constante a X para calcular VIF
X_with_const = sm.add_constant(X_cleaned1)

# Calcular VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]


for i in range(1,len(vif_data)):
    if vif_data["VIF"][i]>5:
        print(vif_data["Variable"][i],vif_data["VIF"][i])


poutcome_unknown 7.0537108370351165


MODELO 1: VARIABLE DE RESPUESTA Y

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import numpy as np

X= X_cleaned1.drop(columns='y_yes')
# Separar variables predictoras (X) y la variable objetivo (y)
y = X_cleaned1['y_yes']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=1)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

In [6]:
# mean absolute error
MAE = metrics.mean_absolute_error(y_test, y_pred)

# mean squared error
MSE = metrics.mean_squared_error(y_test, y_pred)

# root mean squared error
RMSE = np.sqrt(MSE)

print("MAE: ", MAE)
print("MSE: ", MSE)
print("RMSE: ", RMSE)

MAE:  0.15248625954163192
MSE:  0.06555171655623256
RMSE:  0.2560306945587434


In [8]:

# usar MSE - error cuadrático medio
scores = cross_val_score(model, X, y, cv=100, scoring='neg_mean_squared_error')
mse_scores = - scores
scores = cross_val_score(model, X, y, cv=100, scoring='neg_mean_absolute_error')

mae_scores = -scores

#print("MAE scores for each fold:", mae_scores)
print("Mean MAE:", np.mean(mae_scores))

rmse_scores = np.sqrt(mse_scores)

print("Mean MSE:", mse_scores.mean())
print("Mean RMSE:", rmse_scores.mean())



Mean MAE: 0.15487925416217363
Mean MSE: 0.06655064153829034
Mean RMSE: 0.21772616688069169


## MODELO 1: OLS VARIABLE DE RESPUESTA Y

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


y = X_cleaned1['y_yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train = sm.add_constant(X_train)

# Ajustar el modelo de regresión usando mínimos cuadrados ordinarios (OLS)
md = sm.OLS(y_train, X_train).fit()

# Resumen de resultados
print(md.summary())

# Extraer coeficientes y p-valores en un DataFrame
results = pd.DataFrame({
    'Variable': md.params.index,
    'Coeficiente': md.params.values,
    'p-valor': md.pvalues.values
})
# Filtrar variables significativas (usualmente p-valor < 0.01)
significativas = results[results['p-valor'] < 0.1]
no_significativas = results[results['p-valor'] >= 0.1]

# Mostrar resultados
print("Variables Significativas (p-valor < 1%):")
print(significativas)

print("\nVariables No Significativas (p-valor >= 1%):")
print(no_significativas)
no_significativas

### MODELO 1: OLS SOLO CON VARIABLES SIGNIFICATIVAS

In [ ]:
X_significativas = X_cleaned1[significativas['Variable'].values[1:]]  # [1:] para excluir 'const'

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_significativas, X_test_significativas, y_train_significativas, y_test_significativas = train_test_split(X_significativas, y, random_state=1)

# Agregar la constante para el modelo OLS
X_train_significativas = sm.add_constant(X_train_significativas)
X_test_significativas = sm.add_constant(X_test_significativas)

# Ajustar el modelo OLS
md_significativas = sm.OLS(y_train_significativas, X_train_significativas).fit()

# Realizar predicciones sobre el conjunto de prueba
y_pred_significativas = md_significativas.predict(X_test_significativas)

# Calcular MAE, MSE y RMSE
mae = mean_absolute_error(y_test_significativas, y_pred_significativas)
mse = mean_squared_error(y_test_significativas, y_pred_significativas)
rmse = np.sqrt(mse)

# Mostrar los resultados
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(md_significativas.summary())

## Modelo 2 logistico completo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
import statsmodels.api as sm

# Separar variables predictoras (X) y la variable objetivo (y)
X = X_cleaned1.drop(columns='y_yes')
y = X_cleaned1['y_yes']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=1)

# Crear y entrenar el modelo de regresión logística con scikit-learn
model_logi = LogisticRegression(max_iter=1000)
model_logi.fit(X_train, y_train)

# Realizar predicciones
y_predlog = model_logi.predict(X_test)

# Calcular y mostrar métricas de clasificación
accuracy = accuracy_score(y_test, y_predlog)
precision = precision_score(y_test, y_predlog)
recall = recall_score(y_test, y_predlog)
f1 = f1_score(y_test, y_predlog)


# Confusion matrix y reporte de clasificación
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_predlog))

print("\nClassification Report:")
print(classification_report(y_test, y_predlog,target_names=['No acepta', 'Acepta']))

# Obtener un resumen detallado con statsmodels
# Añadir una constante al conjunto de datos
X_with_const = sm.add_constant(X)  # Asegúrate de incluir la constante para el modelo en statsmodels

# Ajustar el modelo de regresión logística con statsmodels
logit_model = sm.Logit(y, X_with_const)
result = logit_model.fit()

# Mostrar el resumen del modelo
print(result.summary())

# Convertir el índice en una columna llamada 'Variable'
summary_df = pd.DataFrame({
    'Variable': result.params.index,
    'Coef': result.params,
    'StdErr': result.bse,
    'Z': result.tvalues,
    'P>|Z|': result.pvalues,
    'Conf. Interval Low': result.conf_int()[0],
    'Conf. Interval High': result.conf_int()[1]
}).reset_index()

# Renombrar la columna que contiene el índice
summary_df = summary_df.drop(columns="index")

# Mostrar el DataFrame con la columna 'Variable'
summary_df

# Filtrar variables significativas (usualmente p-valor < 0.01)
significativaslog = results[results['p-valor'] < 0.1]
no_significativaslog = results[results['p-valor'] >= 0.1]

# Mostrar resultados
print("Variables Significativas (p-valor < 1%):")
print(significativaslog)

print("\nVariables No Significativas (p-valor >= 1%):")
print(no_significativas)
no_significativaslog




### Modelo 2: Logistico significativas

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import statsmodels.api as sm
import numpy as np

# Seleccionar las variables significativas (excluyendo 'const')
X_significativas = X_cleaned1[significativaslog['Variable'].values[1:]]  # [1:] para excluir 'const'
y = X_cleaned1['y_yes']  # Asegúrate de que 'y_yes' sea la variable objetivo

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_significativas, X_test_significativas, y_train_significativas, y_test_significativas = train_test_split(
    X_significativas, y, random_state=1
)

# Agregar la constante para el modelo Logit
X_train_significativas = sm.add_constant(X_train_significativas)
X_test_significativas = sm.add_constant(X_test_significativas)

# Ajustar el modelo de regresión logística
md_significativas = sm.Logit(y_train_significativas, X_train_significativas).fit()

# Realizar predicciones sobre el conjunto de prueba
y_pred_proba = md_significativas.predict(X_test_significativas)

# Convertir probabilidades a clases (0 o 1) usando un umbral de 0.5
y_pred_significativas = [1 if prob >= 0.5 else 0 for prob in y_pred_proba]

# Calcular métricas de clasificación
accuracy = accuracy_score(y_test_significativas, y_pred_significativas)
precision = precision_score(y_test_significativas, y_pred_significativas)
recall = recall_score(y_test_significativas, y_pred_significativas)
f1 = f1_score(y_test_significativas, y_pred_significativas)

# Mostrar los resultados

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_significativas, y_pred_significativas))
print("\nClassification Report:")
print(classification_report(y_test_significativas, y_pred_significativas,target_names=['No acepta', 'Acepta']))

# Mostrar el resumen del modelo logístico
md_significativas.summary()



In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Añadir constante a X para calcular VIF
X_with_const = sm.add_constant(X_cleaned1)

# Calcular VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]


for i in range(1,len(vif_data)):
    if vif_data["VIF"][i]>5:
        print(vif_data["Variable"][i],vif_data["VIF"][i])



## Modelo Redes neuronales

In [ ]:
train = X_cleaned1.sample(frac=0.8, random_state=100)
train.head()

In [ ]:
test = X_cleaned1.drop(train.index)
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
sys.setrecursionlimit(999)

In [ ]:
X_cleaned1.columns

In [ ]:
X_cleaned1.dtypes

In [ ]:
# Definir las columnas categóricas y continuas
categorical_features = []

# Encontrar las columnas dummificadas
for col in X_cleaned1.columns:
    if "_" in col and col != "y_yes":  
        categorical_features.append(col)


continuous_features = [col for col in X_cleaned1.columns if col not in categorical_features and col != "y_yes"]

# Mostrar las listas
print("Categorical dummies:", categorical_features)
print("Continuous features:", continuous_features)

In [ ]:
X = X_cleaned1.drop(columns=['y_yes'])

# Definir la variable objetivo
y = X_cleaned1['y_yes']

# Dividir los datos en conjuntos de entrenamiento y prueba
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización
norm = tf.keras.layers.Normalization()
norm.adapt(np.array(train_X))


redes 1

In [ ]:
# Iniciar el cronómetro
start_time = time.time()

# Definir el modelo de red neuronal profunda con capas densas y Dropout
linear_model_1 = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Capa de salida para regresión
])

# Compilar el modelo
linear_model_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mean_absolute_error',
    metrics=['mae', 'mse']
)

# Entrenamiento del modelo
history_1 = linear_model_1.fit(
    train_X,
    train_y,
    epochs=100,
    validation_split=0.2,
    verbose=0
)

# Medir el tiempo de entrenamiento
elapsed_time_1 = time.time() - start_time

Redes 2

In [ ]:
# Iniciar el cronómetro
start_time = time.time()

# Definir el modelo de red neuronal profunda con capas densas y Dropout
linear_model_2 = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(units=1)  # Capa de salida para regresión
])

# Compilar el modelo
linear_model_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_absolute_error',
    metrics=['mae', 'mse']
)


# Entrenamiento del modelo
history_2 = linear_model_2.fit(
    train_X,
    train_y,
    epochs=100,
    validation_split=0.2,
    verbose=0
)

# Medir el tiempo de entrenamiento
elapsed_time_2 = time.time() - start_time

In [ ]:
# Graficar las métricas y la pérdida en subgráficas
fig, axs = plt.subplots(2, 1, figsize=(10, 15))

# Pérdida (Loss)
axs[0].plot(history_1.history['loss'], label='Pérdida de Entrenamiento', color='blue')
axs[0].plot(history_1.history['val_loss'], label='Pérdida de Validación', color='orange', linestyle='--')
axs[0].set_xlabel('Época')
axs[0].set_ylabel('Pérdida (MAE)')
axs[0].set_title(f'Modelo 1 - Pérdida (Tiempo: {elapsed_time_1:.2f} segundos)')
axs[0].legend()
axs[0].grid(True)

# Error Cuadrático Medio (MSE)
axs[1].plot(history_1.history['mse'], label='MSE de Entrenamiento', color='green')
axs[1].plot(history_1.history['val_mse'], label='MSE de Validación', color='red', linestyle='--')
axs[1].set_xlabel('Época')
axs[1].set_ylabel('MSE')
axs[1].set_title('Error Cuadrático Medio (MSE)')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

print(f"Tiempo de entrenamiento del Modelo 1: {elapsed_time_1:.2f} segundos")

In [ ]:
# Graficar las métricas y la pérdida en subgráficas
fig, axs = plt.subplots(2, 1, figsize=(10, 15))

# Pérdida (Loss)
axs[0].plot(history_2.history['loss'], label='Pérdida de Entrenamiento', color='blue')
axs[0].plot(history_2.history['val_loss'], label='Pérdida de Validación', color='orange', linestyle='--')
axs[0].set_xlabel('Época')
axs[0].set_ylabel('Pérdida (MAE)')
axs[0].set_title(f'Modelo 2 - Pérdida (Tiempo: {elapsed_time_2:.2f} segundos)')
axs[0].legend()
axs[0].grid(True)

# Error Cuadrático Medio (MSE)
axs[1].plot(history_2.history['mse'], label='MSE de Entrenamiento', color='green')
axs[1].plot(history_2.history['val_mse'], label='MSE de Validación', color='red', linestyle='--')
axs[1].set_xlabel('Época')
axs[1].set_ylabel('MSE')
axs[1].set_title('Error Cuadrático Medio (MSE)')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

print(f"Tiempo de entrenamiento del Modelo 1: {elapsed_time_2:.2f} segundos")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Asegurarse de que test_X está en formato de numpy array con tipo float32
test_X = np.array(test_X, dtype=np.float32)

# Obtener los valores reales y las predicciones
y_true = list(test_y)  # Convertir directamente a una lista de valores reales
y_pred = []

# Iterar sobre los datos de prueba para obtener las predicciones
for x in test_X:
    prediction = linear_model_1.predict(np.array([x]))  # Hacer predicción
    y_pred.append(prediction.flatten()[0])  # Aplanar y agregar predicción

# Convertir las predicciones a binario (0 o 1)
y_pred_bin = [1 if pred >= 0.5 else 0 for pred in y_pred]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Calcular métricas
print(classification_report(y_true, y_pred_bin, target_names=['No Aprobado', 'Aprobado']))
print("Matriz de confusión:\n", confusion_matrix(y_true, y_pred_bin))